#### Lecture 5: 变形


##### 5.1 长宽表

什么是长表?什么是宽表?这个概念是对于某一个特征而言的。例如:一个表中把性别存储在某一个列中， 那么它就是关于性别的长表;如果把性别作为列名，列中的元素是某一其他的相关特征数值，那么这个表是 关于性别的宽表

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.DataFrame({'Gender':['F','F','M','M'],'Height':[163, 160, 175, 180]}) # 关于性别的长表

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [3]:
pd.DataFrame({'Height: F':[163, 160],'Height: M':[175, 180]}) #关于性别的宽表

,Height: F,Height: M
0,163,175
1,160,180


然这两张表从信息上是完全等价的，它们包含相同的身高统计数值，只是这些数值的呈现方式不同，而其 呈现方式主要又与性别一列选择的布局模式有关，即到底是以 long 的状态存储还是以 wide 的状态存储。

##### 5.1.1 pivot

pivot是一种典型的长表转化为宽表的函数。pivot有三个参数：索引、列和值

对于一个基本的长变宽的操作而言，最重要的有三个要素，分别是<font color=red>变形后的行索引、需要转到列索引的列， 以及这些列和行索引对应的数值</font>，它们分别对应了 pivot 方法中的<mark>index, columns, values 参数</mark>。新生成表的列索引是columns对应列的 unique 值，而新表的行索引是 index 对应列的 unique 值，而 values 对应了想 要展示的数值列。


In [14]:
# 例如：将存储了张三和李四的语文和数学成绩转换成宽表

df = pd.DataFrame({'Class':[1,1,2,2],
                    'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                    'Subject':['Chinese','Math','Chinese','Math'],
                    'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [15]:
df.pivot(index='Name', columns='Subject', values='Grade')

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


<img src="img/ch5_pivot.png" width="50%">

利用pivot进行变形操作需要满足<font color=red>唯一性</font>的要求，即由于在新表中的行列索引对应了唯一的value ，因此原表中的index和columns对应两个列的<mark>行组合必须唯一</mark>。例如，现在把原表中第二行张三的数学改为语文就会报错，这是由于Name与Subject的组合中两次出现 (”San Zhang”, ”Chinese”) ，从而最后不能够确定到 底变形后应该是填写80分还是75分。

In [2]:
df.loc[1,'Subject']='Chinese'

# df.pivot(index='Name', columns='Subject', values='Grade')

NameError: name 'df' is not defined

<mark>pivot相关的三个参数也允许被设置为列表，这也意味着会返回多级索引。</mark>

例如：下表中六列分别为班级、姓名、测试类型(期中考试和期末考试)、科目、成 绩、排名。

In [3]:
df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                              'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                    'Mid', 'Final', 'Mid', 'Final'],
                   'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                                 'Math', 'Math', 'Math', 'Math'],
                   'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                   'rank':[10, 15, 21, 15, 20, 7, 6, 2]})
df

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


现在想要把测试类型和科目联合组成的四个类别（期中语文、期末语文、期中数学、期末数学）转到列索引，并且同时统计成绩和排名：

In [4]:
pivot_multi = df.pivot(index = ['Class', 'Name'],
                       columns = ['Subject','Examination'],
                       values = ['Grade','rank'])
pivot_multi

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

根据唯一性原则，新表的行索引等价于对`index`中的多列使用`drop_duplicates`，而列索引的长度为`values`中的元素个数乘以`columns`的唯一组合数量（与`index`类似） 。从下面的示意图中能够比较容易地理解相应的操作：

<img src="img/ch5_mulpivot.png" width="35%">


#### 随堂练习

Ex1：美国非法药物数据集 

<font color=red>**现有一份关于美国非法药物的数据集，文件位置：data/Drugs.csv。其中`SubstanceName, DrugReports`分别指药物名称和报告数量：**</font>

a) <font color=red>**将数据转为如下的形式：**</font>

<img src="img/Ex5_1.png" width="50%">


##### 5.1.2. pivot_table

`pivot`的使用依赖于***唯一性条件***，那如果不满足唯一性条件，那么必须通过聚合操作使得相同行列组合对应的多个值变为一个值。例如，张三和李四都参加了两次语文考试和数学考试，按照学院规定，最后的成绩是两次考试分数的平均值，此时就无法通过`pivot`函数来完成。

In [5]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang', 
                              'San Zhang', 'San Zhang',
                              'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                                 'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


`pandas`中提供了`pivot_table`来实现，其中的`aggfunc`参数就是使用的聚合函数。上述场景可以如下写出：

In [6]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean')

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


此外，`pivot_table`具有边际汇总的功能，可以通过设置`margins=True`来实现，其中边际的聚合方式与`aggfunc`中给出的聚合方法一致。下面就分别统计了语文均分和数学均分、张三均分和李四均分，以及总体所有分数的均分：

In [13]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc='mean',
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,85,95.0,90.00
Si Li,75,90.0,82.50
All,80,92.5,86.25


#### 随堂练习

接上述题目，实现以下功能

b) **按`State`分别统计每年的报告数量总和,以及总体数量，其中`State, YYYY`分别为列索引和行索引，分别使用`pivot_table`函数。**

##### 5.1.3. melt 宽表变长表

长宽表只是数据呈现方式的差异，但其包含的信息量是等价的，前面提到了利用`pivot`把长表转为宽表，那么就可以通过相应的逆操作把宽表转为长表，`melt`函数就起到了这样的作用。在下面的例子中，`Subject`以列索引的形式存储，现在想要将其压缩到一个列中。

In [11]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [12]:
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese', 'Math'],
                    var_name = 'Subject',
                    value_name = 'Grade')
df_melted

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


在melt()中，id_vars的地位等价于pivot()函数中的index，value_vars代表想要转换到列上的元素，var_name代表value_vars上的含义，value_name代表value_vars在宽表中列对应元素值的含义。

<img src="img/ch5_melt.png" width="35%">

前面提到了`melt`和`pivot`是一组互逆过程，那么就一定可以通过`pivot`操作把`df_melted`转回`df`的形式：

In [7]:
df_unmelted = df_melted.pivot(index = ['Class', 'Name'],
                              columns='Subject',
                              values='Grade')
df_unmelted 
# # 下面需要恢复索引，并且重命名列索引名称
# df_unmelted = df_unmelted.reset_index().rename_axis(columns={'Subject':''})
# df_unmelted.equals(df)
# df_unmelted

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


#### 随堂练习

接上述题目，实现以下功能

c) **将第a)问中的结果恢复为原表。**

#### 5.2 索引的变形

#### 5.2.1. stack与unstack

前面学习了`swaplevel`或者`reorder_levels`进行索引内部的层交换，下面就要讨论$\color{red}{行列索引之间}$的交换，由于这种交换带来了`DataFrame`维度上的变化，因此属于变形操作。

与上面学到的几种变形函数不同，它们都属于某一列或几列$\color{red}{元素}$和$\color{red}{列索引}$之间的转换，而不是索引之间的转换。

`unstack`函数的作用是把行索引转为列索引，例如下面这个简单的例子：

In [13]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['col_1', 'col_2'])
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [14]:
df.unstack() 

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

`unstack`的主要参数是移动的层号，默认将最内层移动到列索引。

In [15]:
df.unstack(1)

col_1      col_2     
          cat  dog   cat  dog
A big     1.0  NaN   1.0  NaN
  small   NaN  1.0   NaN  1.0
B big     1.0  NaN   1.0  NaN
  small   NaN  1.0   NaN  1.0

unstack也支持同时转化多个层

In [8]:
#支持多层转换
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

类似于`pivot`中的唯一性要求，在`unstack`中必须保证$\color{red}{被转为列索引的行索引层}$和$\color{red}{被保留的行索引层}$构成的组合是唯一的，例如把前两个列索引改成相同的破坏唯一性，那么就会报错：

In [16]:
my_index = df.index.to_list()
my_index[1] = my_index[0]
df.index = pd.Index(my_index)
df

col_1  col_2
A cat big      1.0    1.0
      big      1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [17]:
df.unstack()

ValueError: Index contains duplicate entries, cannot reshape

与`unstack`相反，`stack`的作用就是把列索引的层压入行索引，其用法完全类似。

In [10]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['index_1', 'index_2']).T
df

A          B      
         cat   dog  cat   dog
         big small  big small
index_1  1.0   1.0  1.0   1.0
index_2  1.0   1.0  1.0   1.0

In [11]:
df.stack()

A         B     
               cat  dog  cat  dog
index_1 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0
index_2 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0

#### 随堂练习

接上述题目，实现以下功能

d) **将数据转换为如下格式,并尝试将第0,1,2层行索引转化为列索引，并体验将列索引转化为行索引**

<img src="img/Ex5_3.png" width="35%">



#### 综合练习

某连锁店总共有5个大区仓库，每个大区仓库内有若干个子仓库，在data/Ex/area_storage.csv中记录了2010年到2017年各子区仓库中不同货品的村里，其中Year、First_Area、Child_Area和Type这4列以为一次排序状态。

1. 将数据转化为如下格式：
<img src="img/Ex5_2.png" width="35%">

2. 将第(1)问中的结果恢复为原表

3. 按照First_Area分别统计每年的库存量综合，并分别将First_Area和Year作为列索引和行索引，要求分别使用pivot_table()函数与groupby+unstack()两种不同的策略实现。